# Install and Import Dependencies

In [ ]:
! pip install sentence-transformers

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 2.6 MB 13.0 MB/s 
     |████████████████████████████████| 1.2 MB 60.4 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 3.3 MB 59.8 MB/s 
     |████████████████████████████████| 636 kB 59.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=d23512e056512290cf8e886bdcfeab8fd7406f219459ccb030bbe28570bcfff6
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from google.colab import drive
import os

# Load Data

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir(os.getcwd() + '/drive/MyDrive/Colab Notebooks')

In [ ]:
abstracts = pd.read_csv('abstracts.csv')

In [ ]:
titles = pd.read_csv('papers.csv')
titles.drop_duplicates(subset=['eid'], inplace=True)
drop = ['Unnamed: 0', 'doi', 'afid', 'coverDate', 'publicationName', 'source_id', 'citedby_count']
titles.drop(columns=drop, inplace=True)

In [ ]:
merged = pd.merge(titles, abstracts, how='left', on='eid')
merged.dropna(subset=['description'], inplace=True)

In [ ]:
merged.shape

(767968, 5)

# Load and Encode

In [ ]:
# Load model
model = SentenceTransformer('allenai-specter')

In [ ]:
# Amend paper without a title - used DOI to search Google
merged.at[817948, 'title'] = 'Review and analysis of fuel cell system modelling and control'

In [ ]:
# Encode papers
paper_texts = [row['title'] + '[SEP]' + row['description'] for i, row in merged.iterrows()]

In [ ]:
paper_texts[0]

'Regulation of steroid sulphatase expression and activity in breast cancer[SEP]Steroid sulphatase (STS) catalysis the conversion of oestrone sulphate (E1S) to oestrone (E1) and its action in breast tumours makes a major contribution to in situ oestrogen production in this tissue. Although expression of STS mRNA and STS activity are increased in malignant breast tissues compared with that in non-malignant tissues, little is known about the regulation of its expression or activity. In the present study we have used a RT-PCR technique to investigate the regulation of STS mRNA expression in cultured breast tissue fibroblasts and MCF-7 cells. STS mRNA expression was readily detectable in fibroblasts derived from breast tissue proximal to tumours, breast tumour tissue and reduction mammoplasty tissue. For two pre-menopausal subjects, STS mRNA expression was similar in proximal and tumour fibroblasts whereas for a third, post-menopausal subject, expression in breast tumour fibroblasts was 2.4

In [ ]:
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)

# Search Corpus

In [ ]:
def search_papers(title, abstract):
  query_embedding = model.encode(title+'[SEP]'+abstract, convert_to_tensor=True)

  search_hits = util.semantic_search(query_embedding, corpus_embeddings)
  search_hits = search_hits[0]

  print('Paper:', title)
  print('Most similar papers:')
  print(search_hits)
  for hit in search_hits:
    related_paper = merged.iloc[hit['corpus_id']]
    print(hit['score'], related_paper['title'])
    #print("{:.2f}\t{}\t{} {}".format(hit['score'], related_paper['title'], related_paper['venue'], related_paper['year']))

In [ ]:
search_papers(title='Regulation of steroid sulphatase expression and activity in breast cancer',
              abstract='Steroid sulphatase (STS) catalysis the conversion of oestrone sulphate (E1S) to oestrone (E1) and its action in breast tumours makes a major contribution to in situ oestrogen production in this tissue. Although expression of STS mRNA and STS activity are increased in malignant breast tissues compared with that in non-malignant tissues, little is known about the regulation of its expression or activity. In the present study we have used a RT-PCR technique to investigate the regulation of STS mRNA expression in cultured breast tissue fibroblasts and MCF-7 cells. STS mRNA expression was readily detectable in fibroblasts derived from breast tissue proximal to tumours, breast tumour tissue and reduction mammoplasty tissue. For two pre-menopausal subjects, STS mRNA expression was similar in proximal and tumour fibroblasts whereas for a third, post-menopausal subject, expression in breast tumour fibroblasts was 2.4-fold that in proximal fibroblasts. The cytokine tumour necrosis factor α (TNFα) or the STS inhibitor, 2-methoxyoestrone-3-O-sulphamate, had no effect on STS mRNA expression in fibroblasts. STS mRNA was detectable in MCF-7 cells but neither TNFα nor interleukin 6 (IL-6) affected its expression. Transient transfection of COS-1 and MCF-7 cells with a STS cDNA lacking STS 5′ and 3′ sequences increased activity 17-fold and 2-fold, respectively. TNFα plus IL-6 increased STS activity in mock transfected MCF-7 cells and further increased STS activity in transfected MCF-7 cells. This indicates that activation can occur independently of STS promoter and enhancer elements. In conjunction with the lack of regulation of STS mRNA it suggest that TNFα and IL-6 may increase STS activity via a post-translational modification of the enzyme or by increasing substrate availability. © 2001 Elsevier Science Ltd.')

Paper: Regulation of steroid sulphatase expression and activity in breast cancer
Most similar papers:
[{'corpus_id': 0, 'score': 0.9999999403953552}, {'corpus_id': 2, 'score': 0.7828977704048157}, {'corpus_id': 5, 'score': 0.6106774806976318}, {'corpus_id': 4, 'score': 0.5788747668266296}, {'corpus_id': 8, 'score': 0.5634571313858032}, {'corpus_id': 3, 'score': 0.529052734375}, {'corpus_id': 7, 'score': 0.47814470529556274}, {'corpus_id': 1, 'score': 0.4768742322921753}, {'corpus_id': 9, 'score': 0.4410049021244049}, {'corpus_id': 6, 'score': 0.4281136989593506}]
0.9999999403953552 Regulation of steroid sulphatase expression and activity in breast cancer
0.7828977704048157 Inhibition of steroid sulphatase activity by tricyclic coumarin sulphamates
0.6106774806976318 In vivo analysis of Cajal body movement, separation, and joining in live human cells
0.5788747668266296 Analysing cache effects in distribution sorting
0.5634571313858032 Y chromosome microdeletions in idiopathic azoospermi

# Save Embeddings

In [ ]:
np.save('specter_embeddings.npy', corpus_embeddings.cpu())

In [ ]:
corpus_embeddings.shape

torch.Size([767968, 768])